In [2]:
import os
import pandas as pd
from src.utils import read_config
import google.generativeai as genai
import google.ai.generativelanguage as glm

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


e:\miniconda3\envs\project_envelop\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
organized_result = pd.read_csv('temp_result/organized_result_1.csv')
organized_result.shape

(76, 5)

In [4]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500,
    length_function= len
)

In [6]:
reason_loader = DataFrameLoader(organized_result, page_content_column='reason')
reason_data = reason_loader.load()

target_loader = DataFrameLoader(organized_result, page_content_column='target')
target_data = target_loader.load()

reason_documents = text_splitter.transform_documents(reason_data)
print("已為所有文件進行 chunk ",len(reason_documents),"筆")

已為所有文件進行 chunk  76 筆


In [7]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

e:\miniconda3\envs\project_envelop\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
vector_db_path = "./vector_db/"
chroma_collection_name = "langchain"
if not os.path.exists(vector_db_path):
    os.makedirs(vector_db_path)
    print(f"Created folder: {vector_db_path}")
    
vector_store = FAISS.from_documents(reason_documents, embeddings)
vector_store.save_local(folder_path=vector_db_path)

# load FAISS
vectorstore = FAISS.load_local(folder_path=vector_db_path, allow_dangerous_deserialization=True, embeddings=embeddings)

In [ ]:
# query = "查詢: 行政院原住民委員會的採購案"
# embedding_vector = embeddings.embed_query(query)
# docs = vectorstore.similarity_search_by_vector(embedding_vector, k=1)

# match_content_list = []
# for page in docs:
#     match_content_list.append(page.page_content)
#     print(page.page_content)
#     print(page.metadata["target"])
#     print(page.metadata['document'])

行政院客家委員會、行政院原住民族委員會及僑務委員會將客家電視、原住民電視及臺灣宏觀電視等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，爰依法提案糾正
行政院新聞局
52587.pdf


In [9]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [11]:
def query_similarity_content(query, top_k=1):
    embedding_vector = embeddings.embed_query(query)
    docs = vectorstore.similarity_search_by_vector(embedding_vector, k=top_k)
    reference = {}
    for i, page in enumerate(docs):
        # match_content_list.append(page.page_content)
        reference[i] = {
            "reference_content": page.page_content,
            "metadata": page.metadata
        }


    template = """
    ### INSTRUCTION: 你是一位資深的監察院資料查詢專家。你的目標是解釋以下 REF 資料，用簡潔準確的方式説明其他人瞭解查詢到的相關內容。",
    ### REF: {reference}
    ### USER: {query}
    ### ASSISTANT: """
    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm
    data = {
          "reference": " ".join(reference[0]['reference_content']),
          "query": query,
      }
    return chain, data, reference

query = "行政院原住民委員會的相關資訊"

chain, data, reference = query_similarity_content(query)
for s in chain.stream(data):
    print(s.content, end="", flush=True)
    
print("\n資料來源:", reference[0]['metadata']['document'])

e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


行政院原住民委員會將原住民電視頻道的製播委託給公共電視文化事業基金會辦理。然而，原民會與公共電視之間對於頻道節目的製播立場不同，造成紛爭。監察院糾正行政院新聞局，認為其未積極解決此爭端。
資料來源: 52587.pdf


In [15]:
query = "行政院原住民委員會的相關資訊"

embedding_vector = embeddings.embed_query(query)
docs = vectorstore.similarity_search_by_vector(embedding_vector, k=1)
reference = {}
for i, page in enumerate(docs):
    # match_content_list.append(page.page_content)
    reference[i] = {
        "reference_content": page.page_content,
        "metadata": page.metadata
    }

In [21]:
template = """
### INSTRUCTION: 你是一位資深的監察院資料查詢專家。你的目標是解釋以下 REF 資料，用簡潔準確的方式説明其他人瞭解查詢到的相關內容。",
### REF: {reference}
### USER: {query}
### ASSISTANT: """
prompt = PromptTemplate.from_template(template)
chain = prompt | llm
data = {
        "reference": "".join(reference[0]['reference_content']),
        "query": query,
    }

In [26]:
response = chain.invoke(data)

e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [28]:
response.content

'行政院原住民委員會將原住民電視頻道節目的製播委託給公共電視文化事業基金會辦理。然而，原民會與公視基金會對於頻道節目的製播立場不同，導致紛爭不斷。行政院新聞局未能積極處理此爭端，因此監察院提出糾正案。'

In [ ]:



chain, data, reference = query_similarity_content(query)
for s in chain.stream(data):
    print(s.content, end="", flush=True)
    
print("\n資料來源:", reference[0]['metadata']['document'])